In [56]:
import pandas as pd

prolongations = pd.read_csv('prolongations.csv')
financial_data = pd.read_csv('financial_data.csv')

financial_data.replace('\xa0', '', regex=True, inplace=True)
financial_data.replace(',', '.', regex=True, inplace=True)

In [58]:
managers = prolongations["AM"].unique()

In [59]:
all_months = ['ноябрь 2022', 'декабрь 2022', 'январь 2023', 'февраль 2023',
              'март 2023', 'апрель 2023', 'май 2023', 'июнь 2023', 'июль 2023',
              'август 2023', 'сентябрь 2023', 'октябрь 2023', 'ноябрь 2023',
              'декабрь 2023', 'январь 2024', 'февраль 2024']

months_to_calc_prolongation = ['январь 2023', 'февраль 2023', 'март 2023', 'апрель 2023',
                  'май 2023', 'июнь 2023', 'июль 2023', 'август 2023',
                  'сентябрь 2023', 'октябрь 2023', 'ноябрь 2023', 'декабрь 2023']

In [60]:
# для начала посчитаем коэффициенты пролонгации для каждого менеджера:
# 1) за каждый месяц (2023 год)
# 2) за год (2023 год)

In [61]:
def capitalize_first_letter(string):
    return string[0].upper() + string[1:]

In [62]:
for manager in managers:
    for prolongation_month in months_to_calc_prolongation:
        # получаем все проекты, которые были завершены в предыдущих месяцах
        prolongation_month_index = all_months.index(prolongation_month)
        month0_index = all_months.index(prolongation_month) - 2
        month1_index = all_months.index(prolongation_month) - 1
        month0 = all_months[month0_index]
        month1 = all_months[month1_index]

        manager_completed_projects = prolongations[prolongations['AM'] == manager]
        
        month0_completed_projects = manager_completed_projects[manager_completed_projects['month'] == month0]
        month1_completed_projects = manager_completed_projects[manager_completed_projects['month'] == month1]
        
        k2_up_sum = 0
        k2_down_sum = 0
        
        ids = month0_completed_projects['id'].unique()
        projects_financial_data = financial_data[financial_data['id'].isin(ids)].copy()
        
        for id, project in projects_financial_data.iterrows():
            for check_month in range(0, prolongation_month_index + 1):
                if project[capitalize_first_letter(all_months[check_month])] == "стоп" or project[capitalize_first_letter(all_months[check_month])] == "end":
                    projects_financial_data.drop(id, inplace=True)
        
        required_months = all_months[0: prolongation_month_index + 1]
        required_months = [capitalize_first_letter(month) for month in required_months]
        required_months = ['id'] + required_months

        projects_financial_data = projects_financial_data[required_months]
        print(projects_financial_data)
        
        # print("До")
        # print(projects_financial_data)
        # projects_financial_data = projects_financial_data[~projects_financial_data.apply(lambda row: row.astype(str).str.contains('стоп|end').any(), axis=1)]
        # print("После")
        # print(projects_financial_data)    

        # print(projects_financial_data)
        # projects_financial_data = projects_financial_data.astype(float)
        # print()
        
        projects_financial_data_grouped = projects_financial_data.groupby("id").sum()
        # print(projects_financial_data_grouped)
        

        # print("проекты без стопов:")
        # print(projects_financial_data)
        
            # if float(project[capitalize_first_letter(prev_prev_month)]):
            #     k2_up_sum += float(project[capitalize_first_letter(prev_prev_month)])
            #     k2_down_sum += float(project[capitalize_first_letter(prev_month)])
            #     print(f"Проект {project['id']} в месяце {prev_prev_month} имеет значение {project[capitalize_first_letter(prev_prev_month)]}")
        # for id in prev_prev_month_completed_projects['id']:
        #     projects = financial_data.loc[financial_data['id'] == id]
        #     for check_month in range(0, prev_prev_month_index + 1):
        #         disbursement = projects[capitalize_first_letter(all_months[check_month])]
        #         print(len(disbursement))
        #         print(disbursement)
            # for _, project in projects.iterrows():
            #     if float(project[capitalize_first_letter(prev_month)]):
            #         print(f"Проект {id} в месяце {prev_month} имеет значение {project[capitalize_first_letter(prev_month)]}")
                
# for months in months_to_calc:
#     prev_prev_month = all_months[all_months.index(months) - 2]
#     prev_month = all_months[all_months.index(months) - 1]
# 
#     prev_prev_month_completed_projects = prolongations[prolongations['month'] == prev_prev_month]
#     prev_month_completed_projects = prolongations[prolongations['month'] == prev_month]
    

    Ноябрь 2022 Декабрь 2022 Январь 2023   id
0      36220.00          NaN         NaN   42
4      10000.00          NaN         NaN  665
8        в ноль          NaN         NaN  579
9      92302.00          NaN         NaN  592
10     12340.00          NaN         NaN  592
11    110000.00          NaN         NaN  666
12       в ноль          NaN         NaN  620
83       в ноль     39245.00    44320.00  453
160   237977.00    144650.00   144650.00  600
161    43440.00     30450.00   122570.00  600
217    40200.00     40200.00    40200.00  633
311    42600.00      7450.00         NaN  369
312         NaN          NaN         NaN  369
    Ноябрь 2022 Декабрь 2022 Январь 2023 Февраль 2023   id
18     55100.00     55100.00         NaN     44775.00   31
57          NaN     29127.50         NaN          NaN  720
58          NaN     29127.50         NaN          NaN  720
60     41565.00     29960.00         NaN          NaN  596
62     70650.00     74650.00         NaN          NaN  498
81